In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

driver = webdriver.Chrome()
driver.get('https://vestibular.brasilescola.uol.com.br/banco-de-redacoes')

# Get the essay theme table
theme_table = driver.find_element(By.ID, 'table-temas')

In [3]:
table_rows = theme_table.find_elements(By.TAG_NAME, 'tr')[1:]

themes = []
for row in table_rows:
    a_element = row.find_elements(By.TAG_NAME, 'td')[0].find_element(By.TAG_NAME, 'a')
    link = a_element.get_property('href')
    theme = a_element.text
    date = row.find_elements(By.TAG_NAME, 'td')[1].text
    month = date.split('/')[0]
    year = date.split('/')[1]
    themes.append([theme, link, month, year])

df = pd.DataFrame(themes, columns=['Theme', 'Link', 'Month', 'Year'])
df.to_csv('data/links.csv', index=False)

In [8]:
links = [df['Link'][0]]
for link in links:
    driver.get(link)
    theme_paragraph = driver.find_element(By.CLASS_NAME, 'texto-conteudo').find_element(By.TAG_NAME, 'p').text
    table_corrected = driver.find_element(By.ID, 'redacoes_corrigidas')
    essays_corrected = table_corrected.find_elements(By.TAG_NAME, 'tr')[1:]
    essays_links = []
    for essay, n in zip(essays_corrected, range(len(essays_corrected))):
        a_element = essay.find_elements(By.TAG_NAME, 'td')[0].find_element(By.TAG_NAME, 'a')
        link = a_element.get_property('href')
        number = n
        essays_links.append([theme_paragraph, n, link])
    theme_hash = hash(link)
    df_theme = pd.DataFrame(essays_links, columns=['Proposition', 'Id', 'Link'])
    df_theme.to_csv(f'data/{theme_hash}.csv', index=False)